
Originally created by Chelsey McGowan-Yallop, SAMS-UHI (sa06cm@sams.ac.uk)

Modified by Muriel Dunn for fish mix analysis

This script uses model-predicted TS(f) spectra to train a machine learning
classifier, performs nested cross-validation, applies the classifier to
measured TS(f) spectra and outputs results files.

To use a different classifier, see the list of supported classifiers at:
https://github.com/hyperopt/hyperopt-sklearn and set as clf.

Sometimes the initial hyperparameter configuration selected by HyperOpt in each
split in the outer loop will be unsuccessful and all trials will fail. The
retry decorator forces it to try again until retry_limit is reached.

OUTPUT FILES:
    _NestedCV.pkl contains results of nested cross-validation procedure
    _Predictions.pkl contains measured TS(f) spectra with predicted labels
    _BestParams.pkl contains the optimal hyperparameters for the model
"""

In [12]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator
from sklearn.neighbors import KNeighborsClassifier
from datetime import timedelta
from tenacity import retry, stop_after_attempt

import sys, errno  


In [8]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/Nya-AZKABAN/AZKABAN-fish2022/'
# .CSV FILES FROM ECHOVIEW
ts_SED_path = path+"AtlanticCod_SED.csv" # Path to Echoview TS(f) file
ts_trackavg_path = path+"AtlanticCod_SED.csv" # Path to Echoview TS(f) file
ts_trackavg_path = path+"AtlanticCod_SED.csv" # Path to Echoview TS(f) file
min_range = 1.0 # Apply min range (m) to targets, else None
max_range = 7.0 # Apply max range (m) to targets, else None

# SCATTERING MODEL
#model_path = path+"AZKABAN_ZoopMix_data_newcopepod.feather" # Path to model outputs
#n_models_per_species = 1000 # No. of models per species

# CLASSIFIER
unique_id = '02-01-2023_kNN_AZKABAN' # Unique ID for output file paths
clf = KNeighborsClassifier(unique_id)  # Classifier

# NESTED CROSS-VALIDATION
preprocessing = [] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 10 # No. of times to retry before failing

In [3]:
# -- PUT THE FUN IN FUNCTIONS -------------------------------------------------

def tsf_import(file_path):

    """
    This function imports wideband frequency response .csv files exported from
    Echoview and performs some basic housekeeping.
    
    PARAMETERS:
        file_path: Path to wideband frequency response .csv file from Echoview.
        
    RETURNS:
        df: Pandas dataframe.
    """
    
    def get_datetime(df):
        """
        Convert Echoview timestamps in imported .csv to datetime.
        """
        df['Ping_microseconds'] = df.Ping_milliseconds * 1000
        df['Ping_microseconds'] = [timedelta(microseconds = i) for i in df['Ping_microseconds']]
        df['Datetime'] = df['Ping_date_Ping_time'] + df['Ping_microseconds']
        df.drop(columns=['Ping_date_Ping_time',
                         'Ping_milliseconds',
                         'Ping_microseconds'],
               inplace=True)
        
        return df
    
    transposed_path = file_path[:-4] + '_transposed.csv'
    if os.path.isfile(transposed_path) == True:
        df = pd.read_csv(transposed_path,
                         index_col='Target_index',
                         skiprows=1,
                         skipfooter=2,
                         engine='python',
                         parse_dates=[['Ping_date', 'Ping_time']])
    else:
        df = pd.read_csv(file_path, low_memory=False).T
        file_path = transposed_path
        df.to_csv(path_or_buf=file_path)
        print('A transposed TS(f) file was created at: \n' + str(file_path))    
        df = pd.read_csv(file_path,
                         index_col='Target_index',
                         skiprows=1,
                         skipfooter=2,
                         engine='python',
                         parse_dates=[['Ping_date', 'Ping_time']])
    
    df = get_datetime(df)
    
    return df

In [4]:
@retry(stop=stop_after_attempt(retry_limit))
def nested_cv(X, y, model, n_splits, n_folds, unique_id):
    
    """
    This function performs nested cross-validation with Bayesian hyperparameter
    optimisation. It uses stratified k-fold cross-validation in both the inner
    and outer loops. After each outer loop, it outputs the results to a .pkl
    file. As there is an element of randomness to the optimisation procedure,
    sometimes all trials will fail. If you re-run the script, it will import
    the incomplete .pkl file and try again.
    
    Note that this is a modified version that uses F1 score as the evaluation
    metric. It also calculates class-specific F1 scores and confusion matrices,
    which are added to the output dataframe.
    
    PARAMETERS:
        X: data minus labels
        y: labels
        model: HyperoptEstimator object
        n_splits: # of splits to use in outer K-fold cross-validation
        n_folds: # of folds to use in inner K-fold cross-validation
        unique_id: Unique name string for file output path
    """
    
    cv = StratifiedKFold(n_splits=n_splits,
                         shuffle=True,
                         random_state=42) # Outer CV
    
    i_start = 0
    i_list = []
    results_df = None
    cv_path = unique_id + '_NestedCV.pkl'
        
    if os.path.isfile(cv_path) == True: # If CV is incomplete, resume
        results_df = pd.read_pickle(cv_path)
        i_start = results_df.Outer_fold.max() + 1
        print('Resuming cross-validation from fold ' + str(i_start + 1))
        
    # Generate indices to split data by StratifiedKFold
    # Append indices for each fold to list    
    for tr_i, te_i in cv.split(X,y):
        i_list.append([tr_i, te_i])
    
    # For each fold...
    for i in range(i_start, len(i_list)):
        results_list = []
        print('Beginning fold ' + str(i+1) + ' of ' + str(len(i_list)))
        
        # Split data into training and test tests
        X_train = X.loc[X.index.intersection(i_list[i][0])]
        y_train = y.loc[y.index.intersection(i_list[i][0])]
        X_test = X.loc[X.index.intersection(i_list[i][1])]
        y_test = y.loc[y.index.intersection(i_list[i][1])]

        start = time.time()
        
        # Fit the HyperoptEstimator to training data (optimise model)
        model.fit(X_train,
                  y_train,
                  n_folds=n_folds, # Inner stratified k-fold CV
                  cv_shuffle=True)
        
        end = time.time()
        duration = end - start

        # Use optimised model to predict labels for test data
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred, average='weighted') # Evaluate
        
        # Everything below: formats and/or calculates results for output file
        sorted_labels = np.sort(y_test.unique())
        unweighted_score = f1_score(y_test, y_pred,
                                    average=None,
                                    labels=sorted_labels)
        c_matrix = confusion_matrix(y_test, y_pred,
                                    labels=sorted_labels)

        for trial in range(len(model.trials.trials)):
                if model.trials.trials[trial].get('result').get('status') == 'ok':
                    trial_loss = model.trials.trials[trial].get('result').get('loss')
                    trial_duration = model.trials.trials[trial].get('result').get('duration')
                else:
                    trial_loss = np.nan
                    trial_duration = np.nan
            
                results_list.append([i,
                                     score,
                                     unweighted_score,
                                     le.inverse_transform(sorted_labels),
                                     c_matrix,
                                     duration,
                                     trial,
                                     trial_loss,
                                     trial_duration])
        
        append_df = pd.DataFrame(results_list,
                                 columns=['Outer_fold',
                                          'Outer_score',
                                          'Outer_unweighted_scores',
                                          'Outer_unweighted_score_labels',
                                          'Outer_confusion_matrix',
                                          'Outer_training_duration',
                                          'Trial',
                                          'Trial_loss',
                                          'Trial_duration'])
        if i == i_start:
            if results_df is not None:
                final_df = pd.concat([results_df,
                                      append_df],
                                      ignore_index=True)
            else:
                final_df = append_df
            final_df.to_pickle(cv_path)
        
        else:
            results_df = pd.read_pickle(cv_path)
            final_df = pd.concat([results_df,
                                  append_df],
                                  ignore_index=True)
            final_df.to_pickle(cv_path)
     

In [5]:
def f1_loss(y_true, y_pred):
    """
    Custom loss function for HyperOpt.
    Uses F1 score instead of accuracy score, as the latter is inappropriate
    for multi-class classification.
    """
    return 1.0 - f1_score(y_true, y_pred, average='weighted')
    

In [6]:

def main():    
    # -- IMPORT FILES ---------------------------------------------------------
     
    measured_df = tsf_import(tsf_path) # Measured TS(f)
    model_df = pd.read_feather(model_path) # Modelled TS(f)

    # -- RESTRUCTURE MODEL DATA -----------------------------------------------

    measured_frequency = [float(i) for i in measured_df.columns.values[3:-1]]
    n_model_f_bins = len(model_df.freq.unique()) # No. freq bins in model data
    n_species = len(model_df.spec.unique()) # No. species in model data

    X_list = []
    y_list = []
    start_lim = 0
    stop_lim = n_model_f_bins

    for i in range(n_species * n_models_per_species):
        TS_array = model_df.TS[start_lim:stop_lim].values
        species_label = model_df.spec[start_lim:stop_lim].values[0]

        X_list.append(TS_array)
        y_list.append(species_label)

        start_lim += n_model_f_bins
        stop_lim += n_model_f_bins

    model_df = pd.DataFrame(X_list, columns=measured_frequency)
    model_df['Species'] = y_list

    # -- WRANGLE DATA ---------------------------------------------------------

    le = LabelEncoder() # Maps labels -> int (e.g. Copepods -> 0, Krill -> 1)
    model_df['Species'] = le.fit_transform(model_df.Species)
    X = model_df.iloc[:, :-1] # Features, TS(f) only
    y = model_df.Species # Labels

    if min_range != None:
        measured_df = measured_df[measured_df.Range > min_range]

    if max_range != None:
        measured_df = measured_df[measured_df.Range < max_range]

    measured_X = measured_df.iloc[:, 3:-1] # Features, TS(f) only

    # -- NESTED CROSS-VALIDATION ----------------------------------------------

    model = HyperoptEstimator(classifier = clf,
                              preprocessing = preprocessing,
                              ex_preprocs = ex_preprocessing,
                              algo = tpe.suggest,
                              trial_timeout = timeout,
                              loss_fn = f1_loss,
                              max_evals = max_evals)#,
                              #n_jobs = n_jobs)

    nested_cv(X, y, model, n_splits, n_folds, unique_id)

    # -- RETRAIN MODEL --------------------------------------------------------

    print('Retraining model on full dataset')

    model = HyperoptEstimator(classifier = clf,
                              preprocessing = preprocessing,
                              ex_preprocs = ex_preprocessing,
                              algo = tpe.suggest,
                              trial_timeout = timeout,
                              loss_fn = f1_loss,
                              max_evals = max_evals)#,
                              #n_jobs = n_jobs)

    model.fit(X, y, n_folds=n_folds, cv_shuffle=True)

    # -- PREDICT CLASSES FOR NEW DATA -----------------------------------------

    print('Classifying new data')

    y_pred = model.predict(measured_X) # Predict classes for measured TS(f)
    y_pred = le.inverse_transform(y_pred) # Transform labels back to species

    # -- OUTPUT RESULTS -------------------------------------------------------

    measured_df['Prediction'] = y_pred
    measured_df.to_pickle(unique_id + '_Predictions.pkl')

    with open(unique_id + '_BestParams.pkl', 'wb') as handle:
        pickle.dump(model.best_model(), handle)

    
   

In [9]:
measured_df = tsf_import(tsf_path)

A transposed TS(f) file was created at: 
F:/Nya-AZKABAN/AZKABAN-fish2022/AtlanticCod_SED_transposed.csv


In [10]:
measured_df

,Ping_index,Range,Depth,90.000,90.500,91.000,91.500,92.000,92.500,93.000,...,166.000,166.500,167.000,167.500,168.000,168.500,169.000,169.500,170.000,Datetime
Target_index,,,,,,,,,,,,,,,,,,,,,
0,0,5.628848,5.617238,-31.915427,-32.438779,-32.639425,-32.373674,-32.181812,-32.299312,-32.761376,...,-46.172522,-46.564779,-47.031860,-47.691934,-48.610345,-49.751749,-50.981897,-51.929721,-52.222216,2022-01-13 10:30:20.119
1,0,6.591144,6.571238,-36.114666,-32.214917,-30.416021,-29.992647,-31.018295,-33.340891,-35.952621,...,-50.671039,-56.296144,-67.409275,-63.254305,-61.576541,-63.948518,-64.609738,-65.216180,-69.957655,2022-01-13 10:30:20.119
2,1,5.623051,5.610025,-38.280398,-36.966869,-36.781604,-37.251602,-38.400912,-39.620860,-39.941762,...,-61.052424,-64.412874,-66.698775,-64.882859,-60.389657,-56.257941,-53.426689,-51.985144,-51.928260,2022-01-13 10:30:20.674
3,1,5.640442,5.626242,-36.151157,-36.138085,-36.823339,-37.760591,-38.726831,-39.061169,-38.784957,...,-60.422621,-64.317494,-60.951879,-58.635508,-57.245403,-54.419167,-51.911909,-51.049755,-51.767891,2022-01-13 10:30:20.674
4,1,6.614331,6.592303,-30.269844,-31.348255,-33.214381,-34.694713,-34.609010,-33.484105,-32.724687,...,-39.959937,-40.213835,-41.258542,-43.248557,-45.707007,-46.949574,-46.916999,-47.852483,-51.539544,2022-01-13 10:30:20.674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79385,35617,5.796960,5.795151,-39.339739,-38.727011,-38.249553,-37.873847,-37.692556,-37.701828,-37.709297,...,-37.981680,-38.673042,-39.446617,-40.191212,-40.784936,-41.194848,-41.526568,-41.963395,-42.648993,2022-01-13 14:44:58.812
79386,35618,4.162217,4.161632,-30.626774,-30.016487,-29.644226,-29.524220,-29.748905,-30.245836,-30.668648,...,-37.283316,-37.020772,-37.024834,-37.488932,-38.492096,-39.900057,-41.069759,-41.073801,-40.170357,2022-01-13 14:44:59.255
79387,35618,4.394096,4.391965,-25.452183,-27.662271,-29.283096,-29.702831,-28.553149,-27.898429,-29.118468,...,-35.351394,-36.695058,-40.896311,-51.335559,-43.740377,-38.536115,-36.204435,-35.100141,-34.984296,2022-01-13 14:44:59.255
